In [94]:
from math import exp
from random import seed
from random import random
import numpy as np
import matplotlib.pyplot as plt

# Initialize a network #w
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)] 
    network.append(hidden_layer) 
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer) 
    return network


def activate(weights, inputs):
    z = weights[-1] #bias is the last layer
    for i in range(len(weights)-1):
        z += weights[i] * inputs[i]
    return z

# Sigmoid neuron activation
def sigmoid(z):
    return 1.0 / (1.0 + exp(-z))

# Forward propagate input to a network output
def forward_propagate(network, sample):
    inputs = sample
    for layer in network:
        new_inputs = []
        for neuron in layer:
            z = activate(neuron['weights'], inputs)
            neuron['output'] = sigmoid(z)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs #We work through each layer of 

In [95]:
#backward propagation

In [96]:
def sigmoid_derivative(output): #g'(z)
    return output*(1.0 - output)

In [97]:
def back_propagate_error(network, y):
    for i in reversed(range(len(network))):
        layer=network[i]
        errors=list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error=0.0
                for neuron in network[i + 1]:
                    error=error + neuron['weights'][j] * neuron['delta']
                    errors.append(error)
        else:
            for j in range(len(layer)):
                neuron=layer[j]
                errors.append(y[j] - neuron['output'])
        for j in range(len(layer)):
            neuron=layer[j]
            neuron['delta']=errors[j]*sigmoid_derivative(neuron['output'])

In [98]:
def update_weights(network, sample, alpha):
    for i in range(len(network)):
        inputs=sample[:-1]
        if i != 0:
            inputs=[neuron['output'] for neuron in network[i - 1]]
            for neuron in network[i]:
                for j in range(len(inputs)):
                    neuron['weights'][j] = neuron['weights'][j] + alpha * neuron['delta'] * inputs[j]
                neuron['weights'][-1] = neuron['weights'][-1] + alpha * neuron['delta']

In [99]:
def train_network(network, train, alpha, epochs, n_outputs):
    error_list = []
    for epoch in range(epochs):
        sum_errors=0
        for sample in train:
            outputs=forward_propagate(network, sample)
            y=[0 for i in range(n_outputs)] 
            y[sample[-1]] = 1
            sum_errors = sum_errors + sum([(y[i] - outputs[i]) ** 2 for i in range(len(y))])
            back_propagate_error(network, y)
            update_weights(network, sample, alpha)
        print('epoch=%d, lrate=%.3f, error = %.3f'%(epoch, alpha, sum_errors))
        error_list.append(sum_errors)
    return error_list


In [100]:
import pandas as pd

dataset=pd.read_csv('../dataset/LogisticRegressiondata.csv')
dataset

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0,Tunisia,2016-03-27 00:53:11,0
1,80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,1,Nauru,2016-04-04 01:39:02,0
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,Davidton,0,San Marino,2016-03-13 20:35:42,0
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1,Italy,2016-01-10 02:31:19,0
4,68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,0,Iceland,2016-06-03 03:36:18,0
...,...,...,...,...,...,...,...,...,...,...
995,72.97,30,71384.57,208.58,Fundamental modular algorithm,Duffystad,1,Lebanon,2016-02-11 21:49:00,1
996,51.30,45,67782.17,134.42,Grass-roots cohesive monitoring,New Darlene,1,Bosnia and Herzegovina,2016-04-22 02:07:01,1
997,51.63,51,42415.72,120.37,Expanded intangible solution,South Jessica,1,Mongolia,2016-02-01 17:24:57,1
998,55.55,19,41920.79,187.95,Proactive bandwidth-monitored policy,West Steven,0,Guatemala,2016-03-24 02:35:54,0


In [101]:
dataset = dataset.drop(['Ad Topic Line', 'Timestamp', 'City', 'Country'], axis = 1).values

In [102]:
label = dataset[:,-1]
np.shape(label)

(1000,)

In [103]:
dataset = dataset[:,:-1]
dataset = (dataset - dataset.min()) / (dataset.max() - dataset.min())
dataset = np.insert(dataset, len(dataset[0]), label, axis = 1)
print(dataset)

[[8.67461452e-04 4.40335762e-04 7.77933643e-01 3.22187387e-03
  0.00000000e+00 0.00000000e+00]
 [1.00937538e-03 3.90011675e-04 8.61068406e-01 2.43782459e-03
  1.25810218e-05 0.00000000e+00]
 [8.74003583e-04 3.27106566e-04 7.52168213e-01 2.97541165e-03
  0.00000000e+00 0.00000000e+00]
 ...
 [6.49558155e-04 6.41632111e-04 5.33633097e-01 1.51437759e-03
  1.25810218e-05 1.00000000e+00]
 [6.98875760e-04 2.39039414e-04 5.27406372e-01 2.36460304e-03
  0.00000000e+00 0.00000000e+00]
 [5.66271790e-04 3.27106566e-04 3.75868091e-01 2.24382523e-03
  0.00000000e+00 1.00000000e+00]]


In [104]:
dataset = dataset.tolist()

In [105]:
np.shape(dataset)
print(len(dataset[1]))

6


In [106]:
for row in dataset:
    row[-1] = int(row[-1])

In [107]:
from random import seed
seed(1)
n_inputs=len(dataset[0]) - 1
n_hidden=2
n_outputs=len(set([sample[-1] for sample in dataset]))
network=initialize_network(n_inputs, 2, n_outputs)
alpha=0.03
epochs=10000
initialize_network(n_inputs, n_hidden, n_outputs)

[[{'weights': [0.9014274576114836,
    0.030589983033553536,
    0.0254458609934608,
    0.5414124727934966,
    0.9391491627785106,
    0.38120423768821243]},
  {'weights': [0.21659939713061338,
    0.4221165755827173,
    0.029040787574867943,
    0.22169166627303505,
    0.43788759365057206,
    0.49581224138185065]}],
 [{'weights': [0.23308445025757263, 0.2308665415409843, 0.2187810373376886]},
  {'weights': [0.4596034657377336,
    0.28978161459048557,
    0.021489705265908876]}]]

In [108]:
print(type(dataset))
train_data = dataset[:][:800]
test_data = dataset[:][801:]

<class 'list'>


In [ ]:
errors = train_network(network, train_data, alpha, epochs, n_outputs)
for layer in network:
    print(layer)

epoch=0, lrate=0.030, error = 437.815
epoch=1, lrate=0.030, error = 399.898
epoch=2, lrate=0.030, error = 399.640
epoch=3, lrate=0.030, error = 399.520
epoch=4, lrate=0.030, error = 399.395
epoch=5, lrate=0.030, error = 399.269
epoch=6, lrate=0.030, error = 399.144
epoch=7, lrate=0.030, error = 399.018
epoch=8, lrate=0.030, error = 398.893
epoch=9, lrate=0.030, error = 398.768
epoch=10, lrate=0.030, error = 398.643
epoch=11, lrate=0.030, error = 398.518
epoch=12, lrate=0.030, error = 398.394
epoch=13, lrate=0.030, error = 398.270
epoch=14, lrate=0.030, error = 398.145
epoch=15, lrate=0.030, error = 398.021
epoch=16, lrate=0.030, error = 397.898
epoch=17, lrate=0.030, error = 397.774
epoch=18, lrate=0.030, error = 397.650
epoch=19, lrate=0.030, error = 397.527
epoch=20, lrate=0.030, error = 397.404
epoch=21, lrate=0.030, error = 397.281
epoch=22, lrate=0.030, error = 397.158
epoch=23, lrate=0.030, error = 397.035
epoch=24, lrate=0.030, error = 396.913
epoch=25, lrate=0.030, error = 396.

epoch=207, lrate=0.030, error = 377.212
epoch=208, lrate=0.030, error = 377.119
epoch=209, lrate=0.030, error = 377.025
epoch=210, lrate=0.030, error = 376.932
epoch=211, lrate=0.030, error = 376.839
epoch=212, lrate=0.030, error = 376.746
epoch=213, lrate=0.030, error = 376.653
epoch=214, lrate=0.030, error = 376.560
epoch=215, lrate=0.030, error = 376.468
epoch=216, lrate=0.030, error = 376.375
epoch=217, lrate=0.030, error = 376.283
epoch=218, lrate=0.030, error = 376.191
epoch=219, lrate=0.030, error = 376.099
epoch=220, lrate=0.030, error = 376.007
epoch=221, lrate=0.030, error = 375.915
epoch=222, lrate=0.030, error = 375.823
epoch=223, lrate=0.030, error = 375.732
epoch=224, lrate=0.030, error = 375.640
epoch=225, lrate=0.030, error = 375.549
epoch=226, lrate=0.030, error = 375.458
epoch=227, lrate=0.030, error = 375.367
epoch=228, lrate=0.030, error = 375.276
epoch=229, lrate=0.030, error = 375.186
epoch=230, lrate=0.030, error = 375.095
epoch=231, lrate=0.030, error = 375.005


epoch=412, lrate=0.030, error = 360.757
epoch=413, lrate=0.030, error = 360.689
epoch=414, lrate=0.030, error = 360.621
epoch=415, lrate=0.030, error = 360.553
epoch=416, lrate=0.030, error = 360.485
epoch=417, lrate=0.030, error = 360.417
epoch=418, lrate=0.030, error = 360.350
epoch=419, lrate=0.030, error = 360.282
epoch=420, lrate=0.030, error = 360.215
epoch=421, lrate=0.030, error = 360.148
epoch=422, lrate=0.030, error = 360.081
epoch=423, lrate=0.030, error = 360.014
epoch=424, lrate=0.030, error = 359.947
epoch=425, lrate=0.030, error = 359.880
epoch=426, lrate=0.030, error = 359.813
epoch=427, lrate=0.030, error = 359.747
epoch=428, lrate=0.030, error = 359.680
epoch=429, lrate=0.030, error = 359.614
epoch=430, lrate=0.030, error = 359.547
epoch=431, lrate=0.030, error = 359.481
epoch=432, lrate=0.030, error = 359.415
epoch=433, lrate=0.030, error = 359.349
epoch=434, lrate=0.030, error = 359.283
epoch=435, lrate=0.030, error = 359.217
epoch=436, lrate=0.030, error = 359.152


epoch=616, lrate=0.030, error = 348.840
epoch=617, lrate=0.030, error = 348.790
epoch=618, lrate=0.030, error = 348.740
epoch=619, lrate=0.030, error = 348.691
epoch=620, lrate=0.030, error = 348.641
epoch=621, lrate=0.030, error = 348.592
epoch=622, lrate=0.030, error = 348.542
epoch=623, lrate=0.030, error = 348.493
epoch=624, lrate=0.030, error = 348.444
epoch=625, lrate=0.030, error = 348.395
epoch=626, lrate=0.030, error = 348.346
epoch=627, lrate=0.030, error = 348.297
epoch=628, lrate=0.030, error = 348.248
epoch=629, lrate=0.030, error = 348.199
epoch=630, lrate=0.030, error = 348.150
epoch=631, lrate=0.030, error = 348.102
epoch=632, lrate=0.030, error = 348.053
epoch=633, lrate=0.030, error = 348.005
epoch=634, lrate=0.030, error = 347.956
epoch=635, lrate=0.030, error = 347.908
epoch=636, lrate=0.030, error = 347.859
epoch=637, lrate=0.030, error = 347.811
epoch=638, lrate=0.030, error = 347.763
epoch=639, lrate=0.030, error = 347.715
epoch=640, lrate=0.030, error = 347.667


epoch=825, lrate=0.030, error = 339.903
epoch=826, lrate=0.030, error = 339.867
epoch=827, lrate=0.030, error = 339.830
epoch=828, lrate=0.030, error = 339.794
epoch=829, lrate=0.030, error = 339.758
epoch=830, lrate=0.030, error = 339.721
epoch=831, lrate=0.030, error = 339.685
epoch=832, lrate=0.030, error = 339.649
epoch=833, lrate=0.030, error = 339.613
epoch=834, lrate=0.030, error = 339.577
epoch=835, lrate=0.030, error = 339.540
epoch=836, lrate=0.030, error = 339.504
epoch=837, lrate=0.030, error = 339.469
epoch=838, lrate=0.030, error = 339.433
epoch=839, lrate=0.030, error = 339.397
epoch=840, lrate=0.030, error = 339.361
epoch=841, lrate=0.030, error = 339.325
epoch=842, lrate=0.030, error = 339.289
epoch=843, lrate=0.030, error = 339.254
epoch=844, lrate=0.030, error = 339.218
epoch=845, lrate=0.030, error = 339.183
epoch=846, lrate=0.030, error = 339.147
epoch=847, lrate=0.030, error = 339.112
epoch=848, lrate=0.030, error = 339.076
epoch=849, lrate=0.030, error = 339.041


...

epoch=8955, lrate=0.030, error = 306.493
epoch=8956, lrate=0.030, error = 306.493
epoch=8957, lrate=0.030, error = 306.493
epoch=8958, lrate=0.030, error = 306.492
epoch=8959, lrate=0.030, error = 306.492
epoch=8960, lrate=0.030, error = 306.492
epoch=8961, lrate=0.030, error = 306.492
epoch=8962, lrate=0.030, error = 306.492
epoch=8963, lrate=0.030, error = 306.492
epoch=8964, lrate=0.030, error = 306.492
epoch=8965, lrate=0.030, error = 306.492
epoch=8966, lrate=0.030, error = 306.492
epoch=8967, lrate=0.030, error = 306.491
epoch=8968, lrate=0.030, error = 306.491
epoch=8969, lrate=0.030, error = 306.491
epoch=8970, lrate=0.030, error = 306.491
epoch=8971, lrate=0.030, error = 306.491
epoch=8972, lrate=0.030, error = 306.491
epoch=8973, lrate=0.030, error = 306.491
epoch=8974, lrate=0.030, error = 306.491
epoch=8975, lrate=0.030, error = 306.490
epoch=8976, lrate=0.030, error = 306.490
epoch=8977, lrate=0.030, error = 306.490
epoch=8978, lrate=0.030, error = 306.490
epoch=8979, lrat

epoch=9158, lrate=0.030, error = 306.469
epoch=9159, lrate=0.030, error = 306.469
epoch=9160, lrate=0.030, error = 306.469
epoch=9161, lrate=0.030, error = 306.468
epoch=9162, lrate=0.030, error = 306.468
epoch=9163, lrate=0.030, error = 306.468
epoch=9164, lrate=0.030, error = 306.468
epoch=9165, lrate=0.030, error = 306.468
epoch=9166, lrate=0.030, error = 306.468
epoch=9167, lrate=0.030, error = 306.468
epoch=9168, lrate=0.030, error = 306.468
epoch=9169, lrate=0.030, error = 306.468
epoch=9170, lrate=0.030, error = 306.467
epoch=9171, lrate=0.030, error = 306.467
epoch=9172, lrate=0.030, error = 306.467
epoch=9173, lrate=0.030, error = 306.467
epoch=9174, lrate=0.030, error = 306.467
epoch=9175, lrate=0.030, error = 306.467
epoch=9176, lrate=0.030, error = 306.467
epoch=9177, lrate=0.030, error = 306.467
epoch=9178, lrate=0.030, error = 306.466
epoch=9179, lrate=0.030, error = 306.466
epoch=9180, lrate=0.030, error = 306.466
epoch=9181, lrate=0.030, error = 306.466
epoch=9182, lrat

epoch=9360, lrate=0.030, error = 306.447
epoch=9361, lrate=0.030, error = 306.447
epoch=9362, lrate=0.030, error = 306.447
epoch=9363, lrate=0.030, error = 306.447
epoch=9364, lrate=0.030, error = 306.446
epoch=9365, lrate=0.030, error = 306.446
epoch=9366, lrate=0.030, error = 306.446
epoch=9367, lrate=0.030, error = 306.446
epoch=9368, lrate=0.030, error = 306.446
epoch=9369, lrate=0.030, error = 306.446
epoch=9370, lrate=0.030, error = 306.446
epoch=9371, lrate=0.030, error = 306.446
epoch=9372, lrate=0.030, error = 306.446
epoch=9373, lrate=0.030, error = 306.446
epoch=9374, lrate=0.030, error = 306.445
epoch=9375, lrate=0.030, error = 306.445
epoch=9376, lrate=0.030, error = 306.445
epoch=9377, lrate=0.030, error = 306.445
epoch=9378, lrate=0.030, error = 306.445
epoch=9379, lrate=0.030, error = 306.445
epoch=9380, lrate=0.030, error = 306.445
epoch=9381, lrate=0.030, error = 306.445
epoch=9382, lrate=0.030, error = 306.445
epoch=9383, lrate=0.030, error = 306.445
epoch=9384, lrat

epoch=9559, lrate=0.030, error = 306.427
epoch=9560, lrate=0.030, error = 306.427
epoch=9561, lrate=0.030, error = 306.427
epoch=9562, lrate=0.030, error = 306.427
epoch=9563, lrate=0.030, error = 306.427
epoch=9564, lrate=0.030, error = 306.427
epoch=9565, lrate=0.030, error = 306.427
epoch=9566, lrate=0.030, error = 306.427
epoch=9567, lrate=0.030, error = 306.426
epoch=9568, lrate=0.030, error = 306.426
epoch=9569, lrate=0.030, error = 306.426
epoch=9570, lrate=0.030, error = 306.426
epoch=9571, lrate=0.030, error = 306.426
epoch=9572, lrate=0.030, error = 306.426
epoch=9573, lrate=0.030, error = 306.426
epoch=9574, lrate=0.030, error = 306.426
epoch=9575, lrate=0.030, error = 306.426
epoch=9576, lrate=0.030, error = 306.426
epoch=9577, lrate=0.030, error = 306.426
epoch=9578, lrate=0.030, error = 306.425
epoch=9579, lrate=0.030, error = 306.425
epoch=9580, lrate=0.030, error = 306.425
epoch=9581, lrate=0.030, error = 306.425
epoch=9582, lrate=0.030, error = 306.425
epoch=9583, lrat

epoch=9765, lrate=0.030, error = 306.409
epoch=9766, lrate=0.030, error = 306.409
epoch=9767, lrate=0.030, error = 306.408
epoch=9768, lrate=0.030, error = 306.408
epoch=9769, lrate=0.030, error = 306.408
epoch=9770, lrate=0.030, error = 306.408
epoch=9771, lrate=0.030, error = 306.408
epoch=9772, lrate=0.030, error = 306.408
epoch=9773, lrate=0.030, error = 306.408
epoch=9774, lrate=0.030, error = 306.408
epoch=9775, lrate=0.030, error = 306.408
epoch=9776, lrate=0.030, error = 306.408
epoch=9777, lrate=0.030, error = 306.408
epoch=9778, lrate=0.030, error = 306.407
epoch=9779, lrate=0.030, error = 306.407
epoch=9780, lrate=0.030, error = 306.407
epoch=9781, lrate=0.030, error = 306.407
epoch=9782, lrate=0.030, error = 306.407
epoch=9783, lrate=0.030, error = 306.407
epoch=9784, lrate=0.030, error = 306.407
epoch=9785, lrate=0.030, error = 306.407
epoch=9786, lrate=0.030, error = 306.407
epoch=9787, lrate=0.030, error = 306.407
epoch=9788, lrate=0.030, error = 306.407
epoch=9789, lrat

In [ ]:
network

In [ ]:
n_inputs

In [ ]:
n_outputs

In [ ]:
output = forward_propagate(network, test_data[0])

In [ ]:
max(output)
print(test_data)

In [ ]:
def predict(network, sample):
    outputs=forward_propagate(network, sample)
    return outputs.index(max(outputs))

In [ ]:
for sample in test_data:
    prediction=predict(network, sample)
    print(sample[-1], prediction)

In [ ]:
plt.plot(range(10000), errors)